# Climate app 

In [ ]:
# Dependencies
from flask import Flask, jsonify

import numpy as np
import pandas as pd
import datetime as dt

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# Database Setup
engine = create_engine("sqlite:///Resources/hawaii.sqlite", connect_args={'check_same_thread': False}, echo=True)

# Reflect database into a new model
Base = automap_base()
# Reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

# Flask Setup
app = Flask(__name__)

#Flask Routes
@app.route("/")
def welcome():
    return (
        f"Hawaii Climate Analysis:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/start<br/>"
        f"/api/v1.0/start/end<br/>")

# Precipitation API
@app.route("/api/v1.0/precipitation")
def precipitation():
    year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    precip_query = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= year_ago).all()

    precip = {date: prcp for date, prcp in precip_query}

    return jsonify(precip)

# Stations API
@app.route("/api/v1.0/stations")
def stations():
    station_query = session.query(Station.station, Station.name).all()

    station = list(np.ravel(station_query))

    return jsonify(station)

# Temperature Observations API
@app.route("/api/v1.0/tobs")
def temp_obs():
    year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    tobs_query = session.query(Measurement.date, Measurement.tobs).\
    filter(Measurement.date >= year_ago).all()

    tobs = list(np.ravel(tobs_query))

    return jsonify(tobs)

# Start date API
@app.route("/api/v1.0/<start>")
def temp_start(start):
    query = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
    filter(Measurement.date >= start).all()

    temp = list(np.ravel(query))

    return jsonify(temp)

# Start/End date API
@app.route("/api/v1.0/<start>/<end>")
def date_range(start, end):
    query = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
    filter(Measurement.date >= start).filter(Measurement.date <= end).all()

    temp =list(np.ravel(query))

    return jsonify(temp)

if __name__ == "__main__":
    #app.run(debug=True)
    app.run(host='0.0.0.0', port=81)

2019-04-20 20:06:24,999 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-04-20 20:06:25,000 INFO sqlalchemy.engine.base.Engine ()
2019-04-20 20:06:25,002 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-04-20 20:06:25,003 INFO sqlalchemy.engine.base.Engine ()
2019-04-20 20:06:25,004 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-04-20 20:06:25,005 INFO sqlalchemy.engine.base.Engine ()
2019-04-20 20:06:25,007 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurement")
2019-04-20 20:06:25,007 INFO sqlalchemy.engine.base.Engine ()
2019-04-20 20:06:25,010 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'measurement' AND type = 'table'
2019-04-20 20:06:25,010 INFO sqlalchemy.engine.base.Engine ()
2019-04-20 20:06:25,012 INFO sqlalchemy.

 * Running on http://0.0.0.0:81/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Apr/2019 20:06:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2019 20:06:28] "GET /favicon.ico HTTP/1.1" 404 -


2019-04-20 20:06:37,569 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-20 20:06:37,571 INFO sqlalchemy.engine.base.Engine SELECT measurement.date AS measurement_date, measurement.prcp AS measurement_prcp 
FROM measurement 
WHERE measurement.date >= ?
2019-04-20 20:06:37,572 INFO sqlalchemy.engine.base.Engine ('2016-08-23',)


127.0.0.1 - - [20/Apr/2019 20:06:37] "GET /api/v1.0/precipitation HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2019 20:08:18] "GET / HTTP/1.1" 200 -


2019-04-20 20:08:27,497 INFO sqlalchemy.engine.base.Engine SELECT measurement.date AS measurement_date, measurement.tobs AS measurement_tobs 
FROM measurement 
WHERE measurement.date >= ?
2019-04-20 20:08:27,498 INFO sqlalchemy.engine.base.Engine ('2016-08-23',)


127.0.0.1 - - [20/Apr/2019 20:08:27] "GET /api/v1.0/tobs HTTP/1.1" 200 -
